# Importing Libraries

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

# Read Dataset

### Features

In [6]:
features = pd.read_csv('dataset/spotify_features_data_2023.csv')
features.head(1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.46,0.125,11,-8.094,0,0.0371,0.891,0.0,0.165,0.275,75.639,audio_features,18r5gmlp3vQykXCskNJI0e,spotify:track:18r5gmlp3vQykXCskNJI0e,https://api.spotify.com/v1/tracks/18r5gmlp3vQy...,https://api.spotify.com/v1/audio-analysis/18r5...,89969,5


In [7]:
urls = features[['type','uri','track_href','analysis_url','id']]

In [8]:
features = features.drop(columns=['type','uri','track_href','analysis_url'])

In [21]:
features.time_signature.unique()

array([5, 4, 1, 3, 0], dtype=int64)

### Tracks

In [10]:
tracks = pd.read_csv('dataset/spotify_tracks_data_2023.csv')
tracks.head(1)

,id,track_popularity,explicit
0,02MBhqgR7WeeI4ZfB6Lo0i,0,False


In [11]:
tracks.shape[0]

438938

In [12]:
track_features = tracks.join(features.set_index('id'), on='id')
track_features.isnull().sum()
track_features

,id,track_popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,02MBhqgR7WeeI4ZfB6Lo0i,0,False,0.574,0.669,7.0,-7.877,1.0,0.0375,0.311000,0.000000,0.3910,0.693,116.491,224133.0,4.0
1,5j4M4gLYuUy9PVw0vsXWyq,0,False,0.456,0.874,0.0,-2.387,1.0,0.0519,0.000145,0.732000,0.0887,0.392,178.016,213712.0,4.0
2,46EucLPJpx0QM3SCjzzdP0,0,False,0.923,0.520,5.0,-10.107,1.0,0.0534,0.299000,0.578000,0.1100,0.881,132.046,99909.0,4.0
3,4nK2LvW3osgDT89S38Mcg6,0,False,0.807,0.834,2.0,-6.028,1.0,0.0306,0.067200,0.000824,0.3480,0.941,120.047,80827.0,4.0
4,1Pq37WIQZDG7yNFnY5IbqG,0,False,0.714,0.855,2.0,-8.896,1.0,0.0303,0.008500,0.739000,0.3430,0.971,120.006,80853.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438933,4rXLjWdF2ZZpXCVTfWcshS,93,True,0.847,0.622,1.0,-6.747,0.0,0.0903,0.119000,0.000000,0.2850,0.220,130.001,125040.0,4.0
438934,7MXVkk9YMctZqd1Srtv4MB,94,True,0.679,0.587,7.0,-7.015,1.0,0.2760,0.141000,0.000006,0.1370,0.486,186.003,230453.0,4.0
438935,6uIIdjYTxxpWOyWuVXrKQO,95,True,0.808,0.944,10.0,-4.863,0.0,0.0408,0.183000,0.000001,0.1380,0.937,124.934,191467.0,3.0
438936,4MjDJD8cW7iVeWInc2Bdyj,96,True,0.787,0.621,4.0,-5.009,0.0,0.0680,0.150000,0.000402,0.5800,0.130,139.056,267194.0,4.0


### Artists

In [13]:
artists = pd.read_csv('dataset/spotify_artist_data_2023.csv')
artists.head(10)

,id,name,artist_popularity,artist_genres,followers,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6
0,6GkSLJj5CGLIckLIb8J4LR,Late Night Fights,0,['minneapolis punk'],184,minneapolis punk,NaN,NaN,NaN,NaN,NaN,NaN
1,1N0VwQy5PRJymrRvlRzDIj,Sub Par All Star,0,['antiviral pop'],491,antiviral pop,NaN,NaN,NaN,NaN,NaN,NaN
2,4Ch4BIKKwMJsiu1oVLDwpX,Nirvana Meditation 8D,0,['8d'],56,8d,NaN,NaN,NaN,NaN,NaN,NaN
3,5h2iINTOvhghVcpIzrztPP,Red Hot Rebellion,0,['action rock'],394,action rock,NaN,NaN,NaN,NaN,NaN,NaN
4,7dE2MLL2SaI6MujpU5HFVi,Nirvana Tribute Band,0,['tribute'],230,tribute,NaN,NaN,NaN,NaN,NaN,NaN
5,2cdjmWomWpKyQ0Hqge3dVc,Wyli P,0,['memphis hip hop'],7479,memphis hip hop,NaN,NaN,NaN,NaN,NaN,NaN
6,5g1ztx52qgchXwiRZYnxT4,Caul,1,['dark ambient'],688,dark ambient,NaN,NaN,NaN,NaN,NaN,NaN
7,3NjmnKYEuN92YZiWVsksy7,Chicago Blues All Stars,1,['chicago blues'],730,chicago blues,NaN,NaN,NaN,NaN,NaN,NaN
8,3hqLYpiTCdvz5lG3YHvi4v,Lew Lewis & The Twilight Trio,1,"['neo-rockabilly', 'uk rockabilly']",500,neo-rockabilly,uk rockabilly,NaN,NaN,NaN,NaN,NaN
9,06hu6DFeD2bfdBAts05Vjh,Lower Life Forms,1,['deep east coast hip hop'],169,deep east coast hip hop,NaN,NaN,NaN,NaN,NaN,NaN


### Albums

In [14]:
albums = pd.read_csv('dataset/spotify-albums_data_2023.csv')
albums.head(1)

,track_name,track_id,track_number,duration_ms,album_type,artists,total_tracks,album_name,release_date,label,album_popularity,album_id,artist_id,artist_0,artist_1,artist_2,artist_3,artist_4,artist_5,artist_6,artist_7,artist_8,artist_9,artist_10,artist_11,duration_sec
0,Amazing Grace,6mN8nzCXWUeT5LWEcG1Utx,5,555521,album,NaN,9,Deep Peace Experience,2015-06-09 00:00:00 UTC,Mountain Music Inc,6,3VoHMFQzFr9v163HXRvfo2,002nyxOMwdc9EgNEo7ETcF,Radiancematrix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,555.521


In [15]:
albums.shape[0]

438973

In [16]:
print(albums.track_id.nunique(),albums.shape[0])

438948 438973


In [17]:
track_artists = albums[['track_id','artist_id']]
#track_artists.join(artists.set_index('id'), on='artist_id')

In [18]:
albums = albums.drop(columns=['artist_id','artist_0','artist_1','artist_2','artist_3','artist_4','artist_5','artist_6','artist_7','artist_8','artist_9','artist_10','artist_11','duration_sec','total_tracks','duration_ms','artists'])
albums.head(1)

,track_name,track_id,track_number,album_type,album_name,release_date,label,album_popularity,album_id
0,Amazing Grace,6mN8nzCXWUeT5LWEcG1Utx,5,album,Deep Peace Experience,2015-06-09 00:00:00 UTC,Mountain Music Inc,6,3VoHMFQzFr9v163HXRvfo2


In [19]:
albums.groupby('track_id').count().sort_values('album_id', ascending=False).head(1)

,track_name,track_number,album_type,album_name,release_date,label,album_popularity,album_id
track_id,,,,,,,,
77bNe6jYY8yHdP7orXrz5I,2,2,2,2,2,2,2,2


In [20]:
albums = albums.drop(columns='track_name')
albums

,track_id,track_number,album_type,album_name,release_date,label,album_popularity,album_id
0,6mN8nzCXWUeT5LWEcG1Utx,5,album,Deep Peace Experience,2015-06-09 00:00:00 UTC,Mountain Music Inc,6,3VoHMFQzFr9v163HXRvfo2
1,58sgjtx3FOnBBEOd3BQE6C,8,album,Deep Peace Experience,2015-06-09 00:00:00 UTC,Mountain Music Inc,6,3VoHMFQzFr9v163HXRvfo2
2,45tQJhm0WSk5VWMZK3FsLK,1,album,Deep Peace Experience,2015-06-09 00:00:00 UTC,Mountain Music Inc,6,3VoHMFQzFr9v163HXRvfo2
3,3VFD7QqkJSg0OR6yBwKl7A,6,album,Deep Peace Experience,2015-06-09 00:00:00 UTC,Mountain Music Inc,6,3VoHMFQzFr9v163HXRvfo2
4,7LwJV5BEIC9BQnzvaBSyCc,7,album,Deep Peace Experience,2015-06-09 00:00:00 UTC,Mountain Music Inc,6,3VoHMFQzFr9v163HXRvfo2
...,...,...,...,...,...,...,...,...
438968,2sTDlCxmuZCTDKKk9f1qus,17,album,Nadie Sabe Lo Que Va A Pasar Mañana,2023-10-13 00:00:00 UTC,Rimas Entertainment LLC,100,4FftCsAcXXD1nFO9RFUNFO
438969,6vh4S1z08AjGipGAEsV62e,20,album,Nadie Sabe Lo Que Va A Pasar Mañana,2023-10-13 00:00:00 UTC,Rimas Entertainment LLC,100,4FftCsAcXXD1nFO9RFUNFO
438970,1YnChEM51BVZ5dRhbVFEMl,8,album,Nadie Sabe Lo Que Va A Pasar Mañana,2023-10-13 00:00:00 UTC,Rimas Entertainment LLC,100,4FftCsAcXXD1nFO9RFUNFO
438971,5ixdmTpCmH5diBA1j154wN,22,album,Nadie Sabe Lo Que Va A Pasar Mañana,2023-10-13 00:00:00 UTC,Rimas Entertainment LLC,100,4FftCsAcXXD1nFO9RFUNFO
